In [30]:
import pandas as pd
import numpy as np
import warnings
import openai, os
from openai import OpenAI
import string
import random
import tiktoken
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [31]:
df = pd.read_json(r'../datasets/patch_db.json', encoding='utf_8_sig')
df.dropna(inplace=True)
label2id={'non-security':0,'security':1}
df = df.replace({"category": label2id})
df.rename(columns={'category':'label','diff_code':'diff','commit_message':'message'},inplace=True)
df = df.sample(2000)

C:\Users\15728\AppData\Local\Temp\ipykernel_21080\807549141.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"category": label2id})


In [32]:
id2label={0:'Negative',1:'Positive'}
df.rename(columns={'label':'labels','diff':'diffs','message':'msgs'},inplace=True)
df = df.replace({"labels": id2label})


In [33]:
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [34]:
client = OpenAI(
    api_key="sk-bIHAQMOPc0mMMFKgkvZE9sMyggQvA6zi1SR6oq7uSyZRQpc5",  # 或者 os.environ["OPENAI_API_KEY"]
    base_url="https://sg.uiuiapi.com/v1",  # 注意最好加 /v1,
    # model="gpt-4"
)

In [35]:
def num_tokens_from_messages(messages, model="gpt-4-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-4-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += (
                4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            )
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not presently implemented for model {model}.
  See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )

In [ ]:
# Helper to format the k-shot prompt with:
# - prefix
# - 1 example from each class
# - target text for classification
def generate_promot(row):
    base_promot = """
    Please act as a security patch identificator,
    categorize patch into two categories: Negative, Positive.
    The Negative category corresponds to patches which includes insecure code changes.
    The Positive category corresponds to patches which includes secure code changes.
    I will provide you with the commit message and code diff for a commit patch,
    and you need to give me the category label for this commit patch.
    Please avoid any explanations and only provide the category label.
    """

    commit_message_prompt = f"commit message: {row['msgs']}\n"
    commit_codediff_prompt = f"code diff: {row['diffs']}\n"

    prompt = base_promot + commit_message_prompt + commit_codediff_prompt

    messages = [{"role": "user", "content": prompt}]

    l, r = 0, len(commit_codediff_prompt) - 1
    while l <= r:
        mid = (l + r) // 2
        prompt = base_promot + commit_message_prompt + commit_codediff_prompt[:mid]
        messages = [{"role": "user", "content": prompt}]

        ok = num_tokens_from_messages(messages) < 4096
        if ok:
            l = mid + 1
        else:
            r = mid - 1
    prompt = base_promot + commit_message_prompt + commit_codediff_prompt[:r]

    return prompt

In [37]:
# Helper method to prompt OpenAI LLM and get response.
def get_response(user_content):
    response = client.chat.completions.create(
    model="gpt-4",  # 根据代理方文档修改
    messages=[
        {"role": "user", "content": user_content}
    ]
    )
    print(response.choices[0].message.content)
    
    return response.choices[0].message.content

# text = "\'How can I change my pin?\'"
# examples = get_examples(examples_pool)
# df = pd.read_csv('dataset.csv')

result = []
for i, row in val_df.iterrows():
    user_content = generate_promot(row)
    # print(user_content)
    response =  get_response(user_content)
    result.append(response)
    # print("Model classified ", user_content, " as ", response)


Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Category: Positive
Positive
Positive
Positive
Positive
Positive
Negative
Positive
Positive
Positive
Negative
Negative
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Negative
Positive
Positive
Positive
Positive
Negative
Positive
Positive
Negative
Negative
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Negative
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Negative
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Positive
Negative
Positive
Positive
Positive
Positive
Positive
Negative
Positive
Positive
Positive
Positive
Positive
Positive


In [45]:
val_df['result'] = result

In [46]:
val_df['result']

26404    Positive
3770     Positive
4463     Positive
8353     Positive
3594     Positive
           ...   
9343     Negative
28034    Positive
23044    Positive
8800     Positive
28672    Positive
Name: result, Length: 300, dtype: object

In [39]:
val_df.to_csv('patchDB_result.csv',encoding='utf_8_sig')

In [40]:
# # 仅保留 result 为 'positive' 或 'negative' 的行
# val_df = val_df[val_df['result'].isin(['positive', 'negative'])].copy()

# # 打印对齐检查（行数应一致）
# print(f"After filtering, len(result)={len(val_df['result'])}, len(labels)={len(val_df['labels'])}")

# # 比较结果是否一致
# comparison = val_df['result'] == val_df['labels']

# # 查看不一致的情况
# mismatch_df = val_df[~comparison]

# print(f"共有 {len(mismatch_df)} 行不匹配。")
# display(mismatch_df.head())

In [47]:
val_df['result'] = val_df['result'].replace({'Category: Positive': 'Positive', 'Positive': 'Positive','Negative':'Negative'})

In [50]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(val_df['labels'],val_df['result'],digits=4))

              precision    recall  f1-score   support

    Negative     0.7308    0.0918    0.1631       207
    Positive     0.3139    0.9247    0.4687        93

    accuracy                         0.3500       300
   macro avg     0.5223    0.5083    0.3159       300
weighted avg     0.6015    0.3500    0.2578       300



In [48]:
val_df['result'] 

26404    Positive
3770     Positive
4463     Positive
8353     Positive
3594     Positive
           ...   
9343     Negative
28034    Positive
23044    Positive
8800     Positive
28672    Positive
Name: result, Length: 300, dtype: object

In [49]:
val_df['labels']

26404    Positive
3770     Negative
4463     Negative
8353     Negative
3594     Negative
           ...   
9343     Negative
28034    Positive
23044    Negative
8800     Negative
28672    Positive
Name: labels, Length: 300, dtype: object